# Clean up and add to emission factors from EIA

In [1]:
import pandas as pd

In [2]:
ef = pd.read_excel('EIA CO2 factors.xlsx', header=1, skip_footer=1,
                   index_col='EIA Fuel Code')
ef.columns = [name.strip() for name in ef.columns]
ef['Link'] = 'https://www.eia.gov/electricity/annual/html/epa_a_03.html'

In [3]:
ef.rename_axis({'Factor (Kilograms of CO2 Per Million Btu)**':'Fossil Factor'}, axis=1, inplace=True)

### Add EPA emission factors for fossil fuels not included in the EIA file
https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf
- BFG (blast furnace gas)
- MSW (non-biomass portion)
- OG (other gas, assume fuel gas)

Add zero values for all bio fuels. Still need to figure out what to do for synthetic gas from coal and petroleum...

In [4]:
ef.loc['BFG',:] = ['Blast Furnace Gas', 274.32, None , 'https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf']
ef.loc['MSN',:] = ['Non-biomass Municipal Solid Waste', 90.7, 'Assume the same as MSW', 'https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf']

# Use the bituminous coal factor for synthetic coal
ef.loc['SC',:] = ['Synthetic Coal', 93.3, 'Use same factor as BIT and RC', None]

# Use fuel gas for other gases
ef.loc['OG', :] = ['Other gases', 59, 'Assume fuel gas', 'https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf']

# Use BIT factor for synthetic gas from coal (SGC) and DFO for SGP
# The actual factors won't matter as much because we're going to use the EPA 
# emission values. 
ef.loc['SGC', ['Fossil Factor', 'Notes']] = [93.3, 'Using BIT value. Likely not correct']
ef.loc['SGP', ['Fossil Factor', 'Notes']] = [73.16, 'Using DFO value. Likely not correct']

non_fossil_fuels = ['AB', 'BLQ', 'LFG', 'MSB', 'NUC', 'OBG', 'OBL', 'OBS',
                    'OTH', 'PUR', 'SLW', 'SUN', 'WAT', 'WDL', 'WDS', 'WH', 'WND']

for fuel in non_fossil_fuels:
    ef.loc[fuel, ['Fossil Factor', 'Notes']] = [0, 'non-fossil fuel']

### Add non-fossil emission factors for a total emission factor column

In [5]:
epa_factor_link = 'https://www.epa.gov/sites/production/files/2015-07/documents/emission-factors_2014.pdf'
ipcc_factor_link = 'http://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/2_Volume2/V2_2_Ch2_Stationary_Combustion.pdf'
ef['Total Factor'] = ef['Fossil Factor']
ef.loc['BLQ', ['Total Factor', 'Notes', 'Link']] = [94.4, 'Table 6-6, North American Softwood', 'https://www.epa.gov/sites/production/files/2015-03/documents/subpartaa-tsd-pulp_and_paper.pdf']
ef.loc['AB', ['Total Factor', 'Link']] = [118.17, epa_factor_link]
ef.loc['LFG', ['Total Factor', 'Link']] = [52.17, epa_factor_link]
ef.loc['MSB', ['Total Factor', 'Notes', 'Link']] = [90.7, 'Biomass portion of MSW', epa_factor_link]
ef.loc['OBG', ['Total Factor', 'Link']] = [52.17, epa_factor_link]
ef.loc['OBL', ['Total Factor', 'Link']] = [83.98, ipcc_factor_link]
ef.loc['OBS', ['Total Factor', 'Link']] = [105.51, epa_factor_link]
ef.loc[['SLW', 'WDL'], ['Total Factor', 'Notes', 'Link']] = [83.98, 'Assume same as OBL', ipcc_factor_link]
# ef.loc['WDL', ['Total Factor', 'Notes', 'Link']] = [83.98, 'Assume same as OBL', ipcc_factor_link]
ef.loc['WDS', ['Total Factor', 'Link']] = [93.8, epa_factor_link]

### Export

In [ ]:
path = os.path.join('Clean data', 'Final emission factors.csv')
ef.to_csv(path)

In [6]:
ef

,Fuel,Fossil Factor,Notes,Link,Total Factor
EIA Fuel Code,,,,,
BIT,Bituminous Coal,93.3,NaN,https://www.eia.gov/electricity/annual/html/ep...,93.3
DFO,Distillate Fuel Oil,73.16,NaN,https://www.eia.gov/electricity/annual/html/ep...,73.16
GEO,Geothermal,7.71,NaN,https://www.eia.gov/electricity/annual/html/ep...,7.71
JF,Jet Fuel,70.9,NaN,https://www.eia.gov/electricity/annual/html/ep...,70.9
KER,Kerosene,72.3,NaN,https://www.eia.gov/electricity/annual/html/ep...,72.3
LIG,Lignite Coal,97.7,NaN,https://www.eia.gov/electricity/annual/html/ep...,97.7
MSW,Municipal Solid Waste,41.69,NaN,https://www.eia.gov/electricity/annual/html/ep...,41.69
NG,Natural Gas,53.07,NaN,https://www.eia.gov/electricity/annual/html/ep...,53.07
PC,Petroleum Coke,102.1,NaN,https://www.eia.gov/electricity/annual/html/ep...,102.1
